In [13]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [14]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.13.3
pandas version:   0.20.3
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML file. (Actually, there are several runs that have been concatinated into a YAML file that has a sequence in its top level.)

In [15]:
filename = 'miniGraphics-skybridge-vn-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.DataFrame(yaml_data)

Some of the original runs gave suspect results where the runs took much longer than expected. These appear to be transient issues in the system since a repeat of the run is closer to what is expected. To get more realistic results, I re-ran these conditions. So jettison these runs from the original data and load up the re-runs.

In [16]:
# Remove incorrect readings
data = data.loc[
    ((data['composite-algorithm'] != '2-3-SwapBase') |
     (data['num-processes'] != 4096) |
     (data['image-height'] != 1080)) &
    ((data['composite-algorithm'] != 'BinarySwapFold') |
     (data['num-processes'] != 2896) |
     (data['image-height'] != 1080)) &
    ((data['composite-algorithm'] != 'BinarySwapRemainder') |
     (data['num-processes'] != 2048) |
     (data['image-height'] != 1080)) &
    ((data['composite-algorithm'] != 'BinarySwapTelescoping') |
     (data['num-processes'] != 1616) |
     (data['image-height'] != 1080))
]
# Load correct readings and add them to the data
filename = 'miniGraphics-skybridge-vn-corrections.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.concat([data, pandas.DataFrame(yaml_data)], ignore_index=True)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [17]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [18]:
data = flatten_table(data)

Add a column that gives a human-readable name to each image resolution.

In [19]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data['image-size'] = data['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [20]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'IceTBase': 'IceT'
}

data['composite-algorithm'] = data['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [21]:
import IPython.display

data_description = ''

for column_name in data.columns:
    data_description = data_description + '**' + column_name + '** '
    unique_values = data[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    elif (numpy.issubdtype(unique_values.dtype, numpy.number)):
        data_description = (
            data_description +
            str(numpy.nanmin(unique_values)) + ' &ndash; ' +
            str(numpy.nanmax(unique_values)) + ' '
        )
    elif not pandas.isnull(unique_values).any():
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

**color-buffer-format** byte  
**composite-algorithm** Naive Remainder Telescoping 2-3 Swap IceT nan  
**composite-seconds** 0.00119348 &ndash; 3.49925  
**compress-seconds**  
**construct-tree-seconds** 0.000182052 &ndash; 0.0610641  
**depth-buffer-format** float  
**gather-seconds** 0.00071677 &ndash; 3.45673  
**geometry** box  
**geometry-distribution** duplicate  
**geometry-overlap** -0.05  
**icet-copy-result-seconds**  
**image-compression** True False  
**image-height** 288 1080 4320  
**image-width** 512 1920 7680  
**num-processes** 128 &ndash; 8192  
**num-triangles** 1536 &ndash; 98304  
**paint-seconds** 0.00011823 &ndash; 0.147793  
**painter** simple  
**partial-composite-seconds** 0.000293153 &ndash; 0.86969  
**phi-rotation** -178.406 &ndash; 136.756  
**random-seed** 17627  
**rendering-order-dependent** False  
**start-time** 2018-05-30T02:23:10.000000000 &ndash; 2018-06-08T12:10:45.000000000  
**theta-rotation** -177.945 &ndash; 179.418  
**total-seconds** 0.00146716 &ndash; 3.59942  
**trial-num** 0 &ndash; 19  
**uncompress-seconds**  
**zoom** 1.5  
**image-size** nan HDTV 8K UHD  


## Plot Data

We are plotting for the 2-3 swap algorithm the time it takes to the "partial composite" breaking it up into the time for building the compositing tree and for everything else.

In [22]:
data['transfer-blend-seconds'] = data['partial-composite-seconds'] - data['construct-tree-seconds']

In [23]:
averages = data.pivot_table(
    values=['transfer-blend-seconds', 'construct-tree-seconds'],
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

averages

construct-tree-seconds           transfer-blend-seconds  \
image-size                          8K UHD      HDTV                 8K UHD   
composite-algorithm               2-3 Swap  2-3 Swap               2-3 Swap   
num-processes                                                                 
128                               0.000217  0.000217               0.424664   
160                               0.000246  0.000255               0.440288   
176                               0.000257  0.000273               0.443721   
192                               0.000284  0.000275               0.437828   
224                               0.000336  0.000336               0.437326   
256                               0.000436  0.000421               0.413616   
272                               0.000448  0.000448               0.428039   
320                               0.000505  0.000499               0.425890   
352                               0.000550  0.000545               0.426634   
400                               0.000613  0.000611               0.425138   
448                               0.000691  0.000693               0.421181   
512                               0.000947  0.000900               0.406586   
560                               0.001004  0.000986               0.414580   
640                               0.001145  0.001098               0.409910   
720                               0.001285  0.001250               0.419038   
800                               0.001473  0.001409               0.419721   
912                               0.001744  0.001714               0.407335   
1024                              0.002302  0.002187               0.395628   
1136                              0.002579  0.002622               0.402228   
1280                              0.002908  0.002861               0.403004   
1440                              0.003355  0.003410               0.406350   
1616                              0.003956  0.003981               0.416655   
1824                              0.004690  0.004554               0.417782   
2048                              0.006102  0.005941               0.390473   
2288                              0.006803  0.007112               0.394050   
2576                              0.007993  0.008034               0.397505   
2896                              0.009547  0.009471               0.400676   
3248                              0.011340  0.011801               0.407465   
3648                              0.013720  0.013725               0.395645   
4096                              0.017448  0.017993               0.408855   
4592                              0.020778  0.020940               0.401593   
5152                              0.025024  0.025075               0.395020   
5792                              0.030402  0.030618               0.400436   
6496                              0.037114  0.037382               0.412892   
7296                              0.045518  0.046163               0.393025   
8192                              0.057763  0.059657               0.388839   

                               
image-size               HDTV  
composite-algorithm  2-3 Swap  
num-processes                  
128                  0.023374  
160                  0.024236  
176                  0.023623  
192                  0.023621  
224                  0.024135  
256                  0.022869  
272                  0.023452  
320                  0.023074  
352                  0.022911  
400                  0.023225  
448                  0.023272  
512                  0.022906  
560                  0.022555  
640                  0.022280  
720                  0.022491  
800                  0.022844  
912                  0.022422  
1024                 0.023005  
1136                 0.022535  
1280                 0.026545  
1440                 0.022632  
1616                 0.026062  
1824                 0.0230

Create a filled chart showing the times to construct the composite tree and the rest of the composite.

In [24]:
image_size = 'HDTV'
algorithm = '2-3 Swap'

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

x = averages.index
y = numpy.column_stack((
    numpy.zeros(numpy.shape(x)),
    numpy.array(averages['transfer-blend-seconds', image_size, algorithm]),
    numpy.array(averages['construct-tree-seconds', image_size, algorithm] +
                averages['transfer-blend-seconds', image_size, algorithm]),
))

axes.fill(
    x,
    y,
    color=toyplot.color.Palette()[0],
    opacity=[1.0, 0.6],
)

axes.text(
    1300,
    0.04,
    'Constructing Composite Tree',
    color=toyplot.color.Palette()[0],
    #opacity=0.6,
)

axes.text(
    1300,
    0.01,
    'Remaining Compositing (Transfer/Blending)',
    color='white',
)

<polygon points="45.0,238.0 64.959541883016442,238.0 73.484760355512464,238.0 81.267675044711709,238.0 95.056005167571442,238.0 107.00000000000001,238.0 112.42269615752105,238.0 126.95954188301644,238.0 135.48476035551249,238.0 146.91908376603297,238.0 157.05600516757147,238.0 169.00000000000003,238.0 177.01554705058788,238.0 188.95954188301647,238.0 199.49489197243983,238.0 208.919083766033,238.0 220.63918087821398,238.0 231.00000000000006,238.0 240.28432140929027,238.0 250.95954188301647,238.0 261.49489197243986,238.0 271.80911193061127,238.0 282.63918087821401,238.0 293.00000000000006,238.0 302.91202288026011,238.0 313.51684644310626,238.0 323.99044499915874,238.0 334.25082686548097,238.0 344.63918087821401,238.0 354.99999999999994,238.0 365.22422945389263,238.0 375.51684644310626,238.0 385.99044499915874,238.0 396.25082686548097,238.0 406.63918087821406,238.0 417.0,238.0 417.0,142.75896597267791 406.63918087821406,176.66139403856585 396.25082686548097,176.95032463108009 385.99044499915874,179.30872340329492 375.51684644310626,180.45512992205724 365.22422945389263,181.71798458200442 354.99999999999994,182.48895158571145 344.63918087821401,184.56455537781656 334.25082686548097,185.86534803907114 323.99044499915874,186.18034052867324 313.51684644310626,188.28152757585136 302.91202288026011,188.97003644614941 293.00000000000006,188.28730661601665 282.63918087821401,188.66164643230408 271.80911193061127,182.1870227036722 261.49489197243986,189.53131969902324 250.95954188301647,181.15107541157855 240.28432140929027,189.7399123173887 231.00000000000006,188.73278634178365 220.63918087821398,189.98129994467439 208.919083766033,189.07837444277035 199.49489197243983,189.83366190503949 188.95954188301647,190.28532864101123 177.01554705058788,189.69646757717859 169.00000000000003,188.9447266702052 157.05600516757147,188.16135673482603 146.91908376603297,188.26282145353017 135.48476035551249,188.93426550459492 126.95954188301644,188.58502045012065 112.42269615752105,187.77587301881502 107.00000000000001,189.02509783876667 95.056005167571442,186.31243191159962 81.267675044711709,187.4144471139972 73.484760355512464,187.41021268480807 64.959541883016442,186.09722118397983 45.0,187.94385655644149" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:none"> <polygon points="45.0,187.94385655644149 64.959541883016442,186.09722118397983 73.484760355512464,187.41021268480807 81.267675044711709,187.4144471139972 95.056005167571442,186.31243191159962 107.00000000000001,189.02509783876667 112.42269615752105,187.77587301881502 126.95954188301644,188.58502045012065 135.48476035551249,188.93426550459492 146.91908376603297,188.26282145353017 157.05600516757147,188.16135673482603 169.00000000000003,188.9447266702052 177.01554705058788,189.69646757717859 188.95954188301647,190.28532864101123 199.49489197243983,189.83366190503949 208.919083766033,189.07837444277035 220.63918087821398,189.98129994467439 231.00000000000006,188.73278634178365 240.28432140929027,189.7399123173887 250.95954188301647,181.15107541157855 261.49489197243986,189.53131969902324 271.80911193061127,182.1870227036722 282.63918087821401,188.66164643230408 293.00000000000006,188.28730661601665 302.91202288026011,188.97003644614941 313.51684644310626,188.28152757585136 323.99044499915874,186.18034052867324 334.25082686548097,185.86534803907114 344.63918087821401,184.56455537781656 354.99999999999994,182.48895158571145 365.22422945389263,181.71798458200442 375.51684644310626,180.45512992205724 385.99044499915874,179.30872340329492 396.25082686548097,176.95032463108009 406.63918087821406,176.66139403856585 417.0,142.75896597267791 417.0,15.0 406.63918087821406,77.7991922420821 396.25082686548097,96.893742730769333 385.99044499915874,113.73855796326417 375.51684644310626,126.75548989872846 365.22422945389263,136.87335032700312 354.99999999999994,143.95506773955412 344.63918087821401,155.17177030561234 334.25082686548097,160.59328523080558 323.99044499915874,165.897720250185

In [25]:
toyplot.pdf.render(canvas, '2-3-swap-overhead.pdf')